In [25]:
import pandas as pd
import numpy as np
from siuba import *
# import nltk
# nltk.download('inaugural')
from nltk.corpus import inaugural
import markovify
from hashlib import sha256

In [10]:
def utf8len(s):
    return len(s.encode('utf-8'))

In [11]:
file_ids = inaugural.fileids()
# Create a Markov model for every inaugural address
models = [markovify.Text(inaugural.raw(f)) for f in file_ids]

In [12]:
# Create a model that combines each of the models trained on a single inaugural address
presidential_model = markovify.combine(models)

In [16]:
# Generate 10,000 sentences with < 64 bytes
keepers = list()
i = 0
while i < 1000:
    sentence = presidential_model.make_sentence()
#     import pdb; pdb.set_trace()
    if sentence and utf8len(sentence) <= 64:
        keepers.append({
            'len': utf8len(sentence),
            'sentence': sentence
        })
        i += 1

In [18]:
presidential_df = pd.DataFrame(keepers)
presidential_df

,len,sentence
0,46,"And as we see in it, proposes to paint it out."
1,32,We will again give way to peace.
2,59,I mean loyal: I put out my hand to you Mr. Maj...
3,54,"Washington flourished, but the rectification o..."
4,29,It is a complete sovereignty.
...,...,...
995,47,One of the Declaration of Independence in 1776.
996,41,Doing this I consider the entire country.
997,33,To secure the blessings we enjoy.
998,36,This is to administer for a new age.


In [36]:
hash = sha256('And as we see in it, proposes to paint it out'.encode('utf-8'))
hash.hexdigest()

'287c2f62b10d3dff4d715768da97db18718a7637a554d88fff7e93ed3455f24e'

In [47]:
presidential_df = (
    presidential_df
    >> distinct()
#     >> mutate(sha_256 = _.sentence.str.encode('utf8'))
#     >> mutate(sha_256 = lambda s: sha256(tuple(s.sha_256)))
#     >> mutate(sha_256 = lambda s: s.sentence.encode('utf8'))
)

In [50]:
presidential_df['sha_256'] = [sha256(s.encode('utf-8')).hexdigest() for s in presidential_df['sentence']]
presidential_df

,len,sentence,sha_256
0,46,"And as we see in it, proposes to paint it out.",bf435da99276df72b196ba66bec8b39a8c934cb4ecbf64...
1,32,We will again give way to peace.,ebaf8df63bf2c146cbe0a2743f8558ecced78906f4510f...
2,59,I mean loyal: I put out my hand to you Mr. Maj...,38d16be2c9d7e783ee1878356dcc9eb44087bac0e8f1fd...
3,54,"Washington flourished, but the rectification o...",1c21dc3a14bf9ad73e5ffdfbb33a9b490a5ae628e9c0c3...
4,29,It is a complete sovereignty.,cb9cf620dac9ebd049a00ff0983a27452f1789ef341853...
...,...,...,...
973,47,One of the Declaration of Independence in 1776.,0ae2d1f713357e90f787947ef05f61d1fe2129abccb648...
974,41,Doing this I consider the entire country.,b11420a769fd7a9588f5425c8e4ee8dc7fd2b36cf2a649...
975,33,To secure the blessings we enjoy.,061037bb53582dca8c7ba4e9f9728beecbda1f2668f828...
976,36,This is to administer for a new age.,15cbc803cb7dfd9e6603224cdcd83d40d33e41f0d1089f...


In [55]:
(
    presidential_df
    >> filter(_.sha_256.str.contains('8888'))
)

,len,sentence,sha_256
180,39,Reasoned estimates have been appointed.,257bcbe50ebbab949a24f00a582dbd9059d6b7ae4cf5d4...
754,60,But we Americans know and we will sustain Amer...,d8a6d8affebd88880482674f2e1c7ed6f6548e287183b1...
